In [2]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
filepath = '../data/'


In [3]:
metrics = ['RCV',
           'state_name',
           'city',
           'city_ascii',
           'population',
           'age_median', 
           'income_household_median', 
           'home_value', 
           'rent_median', 
           'education_college_or_above',
           'unemployment_rate',
           'race_white',
           'hispanic']

def read_cities(filename):
    state_cities = pd.read_csv(filename)
    state_cities_filtered = state_cities[metrics]
    return state_cities_filtered

def get_rvn_nonrcv_cities(df):
    rcv_cities = df[df['RCV']==1].reset_index()
    non_rcv_cities = df[df['RCV']!=1].reset_index()
    return rcv_cities, non_rcv_cities
    
#for a given rcv city find the top n most similar non-rcv cities
def find_similar(rcv_city_lookup, rcv_cities_df, non_rcv_cities_df,  n=1):
    rcv_row = rcv_cities_df[rcv_cities_df['city_ascii']==rcv_city_lookup]
    rcv_vec = rcv_row.drop(['RCV', 'city', 'city_ascii'], axis=1)
    cos_sim_scores = np.empty(non_rcv_cities_df.shape[0])
    
    for i, non_rcv_row in non_rcv_cities_df.iterrows():
        non_rcv_vec = non_rcv_row.drop(['RCV', 'city', 'city_ascii'])
        cos_sim = np.dot(rcv_vec, non_rcv_vec) / (norm(rcv_vec)*norm(non_rcv_vec))
        cos_sim_scores[i] = cos_sim
        
    non_rcv_cities_copy = non_rcv_cities_df.copy()
    non_rcv_cities_copy['similarity'] = cos_sim_scores
    non_rcv_cities_copy.sort_values(by=['similarity'], ascending=False, inplace=True)
    return non_rcv_cities_copy.head(n=n)

def print_similar(rcv_cities_df, non_rcv_cities_df, n):
    print("RCV cities:\n", sorted(list(rcv_cities_df['city_ascii'])))
    simi_cities = []
    for city in list(rcv_cities_df['city_ascii']):
        simi_cities = simi_cities + list(find_similar(city, rcv_cities_df, non_rcv_cities_df, n)['city_ascii'])
    print("Non-RCV cities:\n", sorted(simi_cities))
    return list(rcv_cities_df['city_ascii']) + simi_cities


In [4]:
#------ California ------
ca_cities = read_cities(f'{filepath}ca-cities.csv')
ca_cities_filtered = ca_cities.drop(columns = ['state_name'])

ca_cities_filtered.head()

,RCV,city,city_ascii,population,age_median,income_household_median,home_value,rent_median,education_college_or_above,unemployment_rate,race_white,hispanic
0,NaN,Los Angeles,Los Angeles,12815475,35.2,54501,549600.0,2085.0,33.1,8.1,52.2,48.7
1,1.0,San Francisco,San Francisco,3603761,38.3,96265,927729.0,2435.0,55.8,5.4,47.2,15.3
2,NaN,San Diego,San Diego,3210314,34.3,71535,523012.0,1925.0,44.4,7.0,64.7,30.0
3,NaN,Riverside,Riverside,2084749,31.3,62460,318031.0,1556.0,22.5,9.5,61.9,52.8
4,NaN,Sacramento,Sacramento,1854698,34.3,54615,286886.0,1349.0,31.5,9.3,48.5,28.3


In [5]:
#------ California ------
ca_rcv_cities, ca_non_rcv_cities = get_rvn_nonrcv_cities(ca_cities_filtered)

print("-"*10, "RCV", "-"*10)
print(ca_rcv_cities.shape)
print(ca_rcv_cities.head())

print("-"*10, "Non-RCV", "-"*10)
print(ca_non_rcv_cities.shape)
print(ca_non_rcv_cities.head())

find_similar('Oakland', ca_rcv_cities, ca_non_rcv_cities,n=5)

---------- RCV ----------
(7, 13)
   index  RCV           city     city_ascii  population  age_median  \
0      1  1.0  San Francisco  San Francisco     3603761        38.3   
1     12  1.0        Oakland        Oakland      425195        36.4   
2     61  1.0       Berkeley       Berkeley      122324        31.0   
3    101  1.0    San Leandro    San Leandro       90553        40.8   
4    184  1.0    Palm Desert    Palm Desert       52932        52.4   

   income_household_median  home_value  rent_median  \
0                    96265    927729.0       2435.0   
1                    63251    562908.0       2021.0   
2                    75709    861440.0       2218.0   
3                    66178    473702.0       1679.0   
4                    56262    325566.0       1324.0   

   education_college_or_above  unemployment_rate  race_white  hispanic  
0                        55.8                5.4        47.2      15.3  
1                        40.6                8.0        36.7  

,index,RCV,city,city_ascii,population,age_median,income_household_median,home_value,rent_median,education_college_or_above,unemployment_rate,race_white,hispanic,similarity
18,20,NaN,Santa Ana,Santa Ana,334136,31.0,57151,419851.0,1548.0,13.2,6.5,44.2,77.3,0.999544
15,17,NaN,Anaheim,Anaheim,352497,34.0,65313,490973.0,1934.0,25.3,6.7,68.7,53.8,0.999543
19,21,NaN,Santa Rosa,Santa Rosa,321908,38.1,67144,457902.0,1684.0,31.6,6.2,68.8,31.8,0.998974
50,52,NaN,Merced,Merced,144117,29.2,40704,185191.0,1101.0,17.6,16.0,53.3,52.2,0.996510
20,22,NaN,Santa Clarita,Santa Clarita,309378,36.9,90544,460639.0,2173.0,35.0,7.1,70.3,32.3,0.995872


In [6]:
#------ California ------
ca_selected_cities = print_similar(ca_rcv_cities, ca_non_rcv_cities, n = 5)
print(ca_selected_cities)

RCV cities:
 ['Albany', 'Berkeley', 'Eureka', 'Oakland', 'Palm Desert', 'San Francisco', 'San Leandro']
Non-RCV cities:
 ['Alhambra', 'Anaheim', 'Bellflower', 'Buena Park', 'Burbank', 'Calabasas', 'Carpinteria', 'Chino Hills', 'Davis', 'El Paso de Robles', 'Fresno', 'Gilroy', 'Huntington Park', 'Lake Forest', 'Lakewood', 'Livermore', 'Lynwood', 'Merced', 'Montebello', 'Morro Bay', 'Paramount', 'Pico Rivera', 'Pico Rivera', 'Riverside', 'Sacramento', 'San Carlos', 'San Diego', 'San Jose', 'Santa Ana', 'Santa Clarita', 'Santa Rosa', 'Solvang', 'Watsonville', 'Whittier', 'Whittier']
['San Francisco', 'Oakland', 'Berkeley', 'San Leandro', 'Palm Desert', 'Eureka', 'Albany', 'Fresno', 'San Diego', 'Sacramento', 'Riverside', 'San Jose', 'Santa Ana', 'Anaheim', 'Santa Rosa', 'Merced', 'Santa Clarita', 'Alhambra', 'Davis', 'Montebello', 'Burbank', 'Huntington Park', 'Bellflower', 'Watsonville', 'Gilroy', 'Whittier', 'Lynwood', 'Lakewood', 'Pico Rivera', 'Lake Forest', 'Livermore', 'Chino Hills'

In [7]:
#------ Selected states ------
cities_filtered = read_cities(f'{filepath}cities.csv')
cities_filtered.head()


,RCV,state_name,city,city_ascii,population,age_median,income_household_median,home_value,rent_median,education_college_or_above,unemployment_rate,race_white,hispanic
0,NaN,New Mexico,Los Alamos,Los Alamos,12035,41.7,101399,291325.0,1065.0,66.5,3.3,84.4,17.5
1,NaN,New Mexico,Albuquerque,Albuquerque,758523,36.2,49878,188094.0,1063.0,34.3,6.5,73.6,48.5
2,NaN,New Mexico,Rio Rancho,Rio Rancho,96159,38.1,63180,179001.0,1165.0,30.2,7.2,79.9,40.7
3,NaN,New Mexico,Farmington,Farmington,52555,32.9,55179,174799.0,893.0,20.2,6.6,61.1,24.9
4,NaN,New Mexico,North Valley,North Valley,11999,48.1,50236,213404.0,706.0,28.0,4.9,74.9,61.0


In [8]:
all_selected_cities = {"CA":ca_selected_cities}
all_selected_cities

{'CA': ['San Francisco',
  'Oakland',
  'Berkeley',
  'San Leandro',
  'Palm Desert',
  'Eureka',
  'Albany',
  'Fresno',
  'San Diego',
  'Sacramento',
  'Riverside',
  'San Jose',
  'Santa Ana',
  'Anaheim',
  'Santa Rosa',
  'Merced',
  'Santa Clarita',
  'Alhambra',
  'Davis',
  'Montebello',
  'Burbank',
  'Huntington Park',
  'Bellflower',
  'Watsonville',
  'Gilroy',
  'Whittier',
  'Lynwood',
  'Lakewood',
  'Pico Rivera',
  'Lake Forest',
  'Livermore',
  'Chino Hills',
  'Paramount',
  'El Paso de Robles',
  'Pico Rivera',
  'Buena Park',
  'Whittier',
  'Calabasas',
  'Carpinteria',
  'Morro Bay',
  'San Carlos',
  'Solvang']}

In [9]:
cities_filtered.state_name.unique()

array(['New Mexico', 'Colorado', 'Vermont', 'Maine', 'Maryland', 'Utah',
       'Minnesota'], dtype=object)

In [10]:

for state in cities_filtered.state_name.unique():
    print("-"*10, state, "-"*10)
    subset_cities_filtered = cities_filtered[cities_filtered['state_name'] == state]
    subset_cities_filtered = subset_cities_filtered.drop(columns = ['state_name'])
    print("number of cities:", subset_cities_filtered.city.nunique())
    if state == "California":
        state_init = "CA"
    if state == "Colorado":
        state_init = "CO"
    if state == "Maine":
        state_init = "ME"
    if state == "Maryland":
        state_init = "MD"
    if state == "Minnesota":
        state_init = "MN"
    if state == "New Mexico":
        state_init = "NM"
    if state == "Utah":
        state_init = "UT"
    if state == "Vermont":
        state_init = "VT"
    
    if subset_cities_filtered.city_ascii.nunique() < 30:
        print("low number of cities!!!")
        print(subset_cities_filtered.city_ascii.unique())
        all_selected_cities[state_init] = list(subset_cities_filtered.city_ascii.unique())
    elif subset_cities_filtered.RCV.count() < 6: 
        print("low count of RCV!!!")
        n_sim = int(round(30/subset_cities_filtered.RCV.count()))
        print("number of similar non-RCV cities per RCV city:", n_sim)
        subset_rcv_cities, subset_non_rcv_cities = get_rvn_nonrcv_cities(subset_cities_filtered)
        all_selected_cities[state_init] = print_similar(subset_rcv_cities, subset_non_rcv_cities, n = n_sim)
    else:
        subset_rcv_cities, subset_non_rcv_cities = get_rvn_nonrcv_cities(subset_cities_filtered)
        all_selected_cities[state_init] = print_similar(subset_rcv_cities, subset_non_rcv_cities, n = 5)
        
    print("\n")


---------- New Mexico ----------
number of cities: 26
low number of cities!!!
['Los Alamos' 'Albuquerque' 'Rio Rancho' 'Farmington' 'North Valley'
 'Taos' 'Las Cruces' 'Silver City' 'Roswell' 'Lovington' 'Deming'
 'Alamogordo' 'Chaparral' 'Las Vegas' 'Los Lunas' 'Hobbs' 'South Valley'
 'Clovis' 'Sunland Park' 'Artesia' 'Grants' 'Carlsbad' 'Portales' 'Gallup'
 'Espanola' 'Santa Fe']


---------- Colorado ----------
number of cities: 67
low count of RCV!!!
number of similar non-RCV cities per RCV city: 30
RCV cities:
 ['Boulder']
Non-RCV cities:
 ['Alamosa', 'Arvada', 'Brighton', 'Broomfield', 'Canon City', 'Castle Rock', 'Centennial', 'Cherry Creek', 'Commerce City', 'Durango', 'Englewood', 'Fountain', 'Glenwood Springs', 'Golden', 'Greenwood Village', 'Highlands Ranch', 'Ken Caryl', 'Lafayette', 'Littleton', 'Longmont', 'Louisville', 'Loveland', 'Montrose', 'Northglenn', 'Parker', 'Sherrelwood', 'Silverthorne', 'Steamboat Springs', 'Wheat Ridge', 'Windsor']


---------- Vermont -------

In [11]:
all_selected_cities

{'CA': ['San Francisco',
  'Oakland',
  'Berkeley',
  'San Leandro',
  'Palm Desert',
  'Eureka',
  'Albany',
  'Fresno',
  'San Diego',
  'Sacramento',
  'Riverside',
  'San Jose',
  'Santa Ana',
  'Anaheim',
  'Santa Rosa',
  'Merced',
  'Santa Clarita',
  'Alhambra',
  'Davis',
  'Montebello',
  'Burbank',
  'Huntington Park',
  'Bellflower',
  'Watsonville',
  'Gilroy',
  'Whittier',
  'Lynwood',
  'Lakewood',
  'Pico Rivera',
  'Lake Forest',
  'Livermore',
  'Chino Hills',
  'Paramount',
  'El Paso de Robles',
  'Pico Rivera',
  'Buena Park',
  'Whittier',
  'Calabasas',
  'Carpinteria',
  'Morro Bay',
  'San Carlos',
  'Solvang'],
 'NM': ['Los Alamos',
  'Albuquerque',
  'Rio Rancho',
  'Farmington',
  'North Valley',
  'Taos',
  'Las Cruces',
  'Silver City',
  'Roswell',
  'Lovington',
  'Deming',
  'Alamogordo',
  'Chaparral',
  'Las Vegas',
  'Los Lunas',
  'Hobbs',
  'South Valley',
  'Clovis',
  'Sunland Park',
  'Artesia',
  'Grants',
  'Carlsbad',
  'Portales',
  'Gallup